### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [12]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas Data Frames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
school_data_complete.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the overall passing rate (overall average score), i.e. (avg. math score + avg. reading score)/2

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [13]:
total_schools = len(school_data["School ID"].unique())
total_students = len(student_data["Student ID"].unique())
total_budget = school_data["budget"].sum()
average_math_score = student_data["math_score"].mean()
average_reading_score = student_data["reading_score"].mean()

bins = [0,69,100]
group_names = ["Fail","Pass"]

school_data_complete["passed_math"] = pd.cut(school_data_complete["math_score"],bins,labels=group_names)
school_data_complete["passed_reading"] = pd.cut(school_data_complete["reading_score"],bins,labels=group_names)

passing_math = school_data_complete["passed_math"].value_counts()["Pass"] / school_data_complete["passed_math"].value_counts().sum()
passing_reading = school_data_complete["passed_reading"].value_counts()["Pass"] / school_data_complete["passed_reading"].value_counts().sum()


final_results = pd.DataFrame({
    "Total Schools":total_schools,
    "Total Students":total_students,
    "Total Budget":total_budget,
    "Average Math Score":average_math_score,
    "Average Reading Score":average_reading_score,
    "% Passing Math":100 * passing_math,
    "% Passing Reading":100 * passing_reading,
    "% Overall Passing Rate":(average_math_score+average_reading_score)/2
},index=[0])

final_results["Total Students"] = final_results["Total Students"].map("{:,}".format)
final_results["Total Budget"] = final_results["Total Budget"].map("${:,.2f}".format)

final_results.head()


,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
0,15,"39,170","$24,649,428.00",78.985371,81.87784,74.980853,85.805463,80.431606


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)
  
* Create a dataframe to hold the above results

## Top Performing Schools (By Passing Rate)

* Sort and display the top five schools in overall passing rate

In [16]:
schools_grouped = school_data_complete.groupby('school_name')
average_math = schools_grouped["math_score"].mean()
average_reading = schools_grouped["reading_score"].mean()

average_scores = pd.merge(average_math,average_reading,on="school_name")

passed_math = school_data_complete.loc[school_data_complete["math_score"]>=70]
passed_math_grouped = passed_math.groupby('school_name')

passed_reading = school_data_complete.loc[school_data_complete["reading_score"]>=70]
passed_reading_grouped = passed_reading.groupby('school_name')

passed_per_school = pd.merge(passed_math_grouped.count()["Student ID"],passed_reading_grouped.count()["Student ID"],on="school_name")

school_data_averages = pd.merge(school_data,average_scores,on="school_name")
complete_school_data = pd.merge(school_data_averages,passed_per_school,on="school_name")

complete_school_data["Per Student Budget"] = complete_school_data["budget"] / complete_school_data["size"]
complete_school_data["% Passing Math"] = 100 * complete_school_data["Student ID_x"] / complete_school_data["size"]
complete_school_data["% Passing Reading"] = 100 * complete_school_data["Student ID_y"] / complete_school_data["size"]
complete_school_data["% Overall Passing Rate"] = (complete_school_data["% Passing Math"] + complete_school_data["% Passing Reading"])/2

complete_school_data = complete_school_data.rename(columns={"school_name":"School Name",
                                                            "type":"School Type",
                                                            "size":"Total Students",
                                                            "budget":"Total School Budget",
                                                            "math_score":"Average Math Score",
                                                            "reading_score":"Average Reading Score"
                                                            })

complete_school_data = complete_school_data[["School Name","School Type","Total Students","Total School Budget","Per Student Budget","Average Math Score","Average Reading Score","% Passing Math","% Passing Reading","% Overall Passing Rate"]]
complete_school_data = complete_school_data.set_index("School Name")

complete_school_data["Total Students"] = complete_school_data["Total Students"].map("{:,}".format)
complete_school_data["Total School Budget"] = complete_school_data["Total School Budget"].map("${:,.2f}".format)
complete_school_data["Per Student Budget"] = complete_school_data["Per Student Budget"].map("${:,.2f}".format)

complete_school_data.sort_values("% Overall Passing Rate").head(5)


,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Name,,,,,,,,,
Rodriguez High School,District,"3,999","$2,547,363.00",$637.00,76.842711,80.744686,66.366592,80.220055,73.293323
Figueroa High School,District,"2,949","$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,73.363852
Huang High School,District,"2,917","$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,73.500171
Johnson High School,District,"4,761","$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,73.639992
Ford High School,District,"2,739","$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,73.804308


## Bottom Performing Schools (By Passing Rate)

* Sort and display the five worst-performing schools

In [ ]:
complete_school_data.sort_values("% Overall Passing Rate",ascending=True).head(5)

## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

## Reading Score by Grade 

* Perform the same operations as above for reading scores

## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [ ]:
# Sample bins. Feel free to create your own bins.
spending_bins = [0, 585, 615, 645, 675]
group_names = ["<$585", "$585-615", "$615-645", "$645-675"]

## Scores by School Size

* Perform the same operations as above, based on school size.

In [ ]:
# Sample bins. Feel free to create your own bins.
size_bins = [0, 1000, 2000, 5000]
group_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

## Scores by School Type

* Perform the same operations as above, based on school type.